<a href="https://colab.research.google.com/github/LRLeite/Data-Analytics/blob/main/Projeto%20IMDb/IMDb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **🎬 Análise das Avaliações de Filmes e Séries, Segundo o IMDb**

Este notebook foi desenvolvido para realizar uma análise sobre os filmes e séries avaliados no IMDb. O objetivo central é analisar as avaliações do público sobre filmes e séries.<br>



## **🎯 Objetivos da Análise**

Ao longo deste notebook, as seguintes perguntas foram abordadas e respondidas:

* Qual a quantidade de títulos por categoria (Filmes, Séries de TV)?

* Quais são os TOP 10 Filmes com as melhores avaliações?

* Quais são os TOP 10 Filmes com as piores avaliações?

* Quais são as TOP 10 Séries de TV com as melhores avaliações?

* Quais são as TOP 10 Séries de TV com as piores avaliações?

* Quais filmes possuem a maior duração?
<br><br>

**Fonte dos dados:** [IMDB](https://datasets.imdbws.com/)<br>
**Informações sobre os dados:** https://www.imdb.com/interfaces/

# **Bibliotecas**

In [1]:
import pandas as pd
import gzip
import numpy as np

# **Importar os dados**

In [2]:
#Salvar o link dos dados das informações básicas num objeto
url_basics = 'https://datasets.imdbws.com/title.basics.tsv.gz'

#Importar o dataset com os dados das informações básicas
basics = pd.read_csv(url_basics, compression = 'gzip', sep = '\t', usecols = ['tconst','titleType', 'originalTitle', 'startYear', 'endYear', 'runtimeMinutes', 'genres'])
basics.head()

,tconst,titleType,originalTitle,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,1892,\N,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,1893,\N,1,Short


In [3]:
#Verificar estrutura dos dados
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11729243 entries, 0 to 11729242
Data columns (total 7 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   originalTitle   object
 3   startYear       object
 4   endYear         object
 5   runtimeMinutes  object
 6   genres          object
dtypes: object(7)
memory usage: 626.4+ MB


In [4]:
#Salvar o link dos dados das avaliações num objeto
url_ratings = 'https://datasets.imdbws.com/title.ratings.tsv.gz'

#Importar o dataset com os dados das avaliações
ratings = pd.read_csv(url_ratings, compression = 'gzip', sep = '\t')
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2163
1,tt0000002,5.5,296
2,tt0000003,6.5,2217
3,tt0000004,5.3,189
4,tt0000005,6.2,2955


In [5]:
#Verificar estrutura dos dados
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1580294 entries, 0 to 1580293
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1580294 non-null  object 
 1   averageRating  1580294 non-null  float64
 2   numVotes       1580294 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 36.2+ MB


# **Preparar os datasets**

In [6]:
#Verificar a quantidade de valores únicos em ratings
ratings.tconst.nunique()

1580294

In [7]:
#Verificar a quantidade de títulos únicos em basics
basics.originalTitle.nunique()

5308082

In [8]:
#Verificar se existem 'tconst' duplicados em basics
basics[basics['tconst'].duplicated(keep = False)]

,tconst,titleType,originalTitle,startYear,endYear,runtimeMinutes,genres


In [9]:
#Verificar se existem ´tconst´ duplicados em ratings
ratings[ratings['tconst'].duplicated(keep = False)]

,tconst,averageRating,numVotes


Isso mostra que nem todos os títulos possuem avaliação (existem 1.580.294 avaliações contra 5.308.082 de títulos).<br>
Para os propósitos desta análise, os títulos e as avaliações serão utilizados. Dessa forma, utilizarei o inner join para juntar os datasets.

## **Realizar join das tabelas para visualizar as avaliações de cada título**

In [10]:
#Verificar os tipos de conteúdos no dataset, para selecionar apenas filmes e séries para análise
basics.titleType.unique()

array(['short', 'movie', 'tvShort', 'tvMovie', 'tvEpisode', 'tvSeries',
       'tvMiniSeries', 'tvSpecial', 'video', 'videoGame', 'tvPilot'],
      dtype=object)

In [11]:
#Join das tabelas, incluindo "title", "type", "year", "runtime" dos filmes e séries de TV com suas respectivas avaliações
# Alterado de 'left' para 'inner' para realizar um inner join
df_merge = pd.merge(basics[['tconst', 'titleType', 'originalTitle', 'runtimeMinutes']].loc[basics.titleType.isin(['movie', 'tvMovie', 'tvSeries'])], ratings[['tconst','averageRating', 'numVotes']], on = 'tconst', how = 'inner')
df_merge.head()

,tconst,titleType,originalTitle,runtimeMinutes,averageRating,numVotes
0,tt0000009,movie,Miss Jerry,45,5.4,226
1,tt0000147,movie,The Corbett-Fitzsimmons Fight,100,5.3,562
2,tt0000502,movie,Bohemios,100,3.6,22
3,tt0000574,movie,The Story of the Kelly Gang,70,6.0,1002
4,tt0000591,movie,L'enfant prodigue,90,5.4,33


In [12]:
#Verificar valores ausentes
df_merge.isna().sum()

,0
tconst,0
titleType,0
originalTitle,1
runtimeMinutes,0
averageRating,0
numVotes,0


In [13]:
#Verificar NA
df_merge[df_merge['originalTitle'].isna()]

,tconst,titleType,originalTitle,runtimeMinutes,averageRating,numVotes
438898,tt5774076,movie,NaN,\N,7.3,9


In [14]:
#Como o título será utilizado nas análises e essa observação não possui, irei remover do dataset
df_merge = df_merge.dropna(subset = ['originalTitle'])

In [15]:
#Verificar a estrutura dos dados
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Index: 491541 entries, 0 to 491541
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          491541 non-null  object 
 1   titleType       491541 non-null  object 
 2   originalTitle   491541 non-null  object 
 3   runtimeMinutes  491541 non-null  object 
 4   averageRating   491541 non-null  float64
 5   numVotes        491541 non-null  int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 26.3+ MB




Conforme informado no site do [IMDb](https:www.imdb.com/interfaces/), "Um '\\N' é usado para indicar que um determinado campo está ausente ou nulo para esse título/nome". Além disso, as colunas 'startYear', 'endYear', 'runtimeMinutes' são do tipo 'object', o que impossibilita análises númericas que serão realizadas na sequência. Dessa forma, para converter as colunas para tipo númerico na sequência, é necessário substituir os valores '\\N' por nan



In [16]:
#Verificar se existe '\N' em alguma coluna do dataset e contar
total_n_count = 0
for col in df_merge.columns:
    # Convert column to string type before checking
    col_as_str = df_merge[col].astype(str)
    n_count = (col_as_str == '\\N').sum()
    if n_count > 0:
         print(f"A coluna '{col}' contém {n_count} valores exatamente iguais a '\\N'")
         total_n_count += n_count
    else:
        print(f"Não tem nenhum valor igual a '\\N' em {col}")

Não tem nenhum valor igual a '\N' em tconst
Não tem nenhum valor igual a '\N' em titleType
Não tem nenhum valor igual a '\N' em originalTitle
A coluna 'runtimeMinutes' contém 90574 valores exatamente iguais a '\N'
Não tem nenhum valor igual a '\N' em averageRating
Não tem nenhum valor igual a '\N' em numVotes


In [17]:
#Substituir '\\N' por NaN
df_merge = df_merge.replace('\\N', np.nan)

In [18]:
#Converter colunas para tipo Int64
df_merge[['runtimeMinutes', 'numVotes']] = df_merge[['runtimeMinutes', 'numVotes']].astype('Int64')
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Index: 491541 entries, 0 to 491541
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          491541 non-null  object 
 1   titleType       491541 non-null  object 
 2   originalTitle   491541 non-null  object 
 3   runtimeMinutes  400967 non-null  Int64  
 4   averageRating   491541 non-null  float64
 5   numVotes        491541 non-null  Int64  
dtypes: Int64(2), float64(1), object(3)
memory usage: 27.2+ MB




## **Análises**



## **Quantidade de títulos por categoria**

In [19]:
basics[['originalTitle', 'titleType']].groupby('titleType').count()

,originalTitle
titleType,
movie,718106
short,1067307
tvEpisode,9026922
tvMiniSeries,62745
tvMovie,151263
tvPilot,1
tvSeries,283460
tvShort,10649
tvSpecial,52969


## **TOP 10 Filmes com as melhores avaliações**
(25000 votos é o valor mínimo [necessário](https://en.wikipedia.org/wiki/IMDb) para estar listado entre os melhores no IMDb)

In [20]:
#Filmes
df_merge[['originalTitle', 'averageRating', 'numVotes']].loc[(df_merge.titleType == 'movie') & (df_merge.numVotes >= 25000 )].sort_values(by = 'averageRating', ascending = False).head(10)

,originalTitle,averageRating,numVotes
72630,The Shawshank Redemption,9.3,3058122
39835,The Godfather,9.2,2134802
124836,Hababam Sinifi,9.2,44615
183865,The Dark Knight,9.0,3032926
24991,12 Angry Men,9.0,931999
70408,Schindler's List,9.0,1530295
95354,The Lord of the Rings: The Return of the King,9.0,2086345
42179,The Godfather Part II,9.0,1436350
125268,Tosun Pasa,8.9,25614
79300,The Lord of the Rings: The Fellowship of the Ring,8.9,2116817


In [21]:
#Filmes de TV
df_merge[['originalTitle', 'averageRating', 'numVotes']].loc[(df_merge.titleType == 'tvMovie') & (df_merge.numVotes >= 25000 )].sort_values(by = 'averageRating', ascending = False).head(10)

,originalTitle,averageRating,numVotes
233801,David Attenborough: A Life on Our Planet,8.9,35451
32271,A Charlie Brown Christmas,8.3,47457
33287,How the Grinch Stole Christmas!,8.3,63017
243908,Temple Grandin,8.2,34960
31882,Rudolph the Red-Nosed Reindeer,8.0,40212
56954,Threads,7.9,25083
294025,The Normal Heart,7.9,38804
38501,Duel,7.6,82781
129911,Conspiracy,7.6,27570
224509,You Don't Know Jack,7.5,31266


## **TOP 10 Filmes com as piores avaliações**


In [22]:
#Filmes
df_merge[['originalTitle', 'averageRating', 'numVotes']].loc[(df_merge.titleType == 'movie') & (df_merge.numVotes >= 25000 )].sort_values(by = 'averageRating').head(10)

,originalTitle,averageRating,numVotes
442699,Reis,1.0,74357
461376,Cumali Ceber,1.0,39592
443519,Smolensk,1.2,40357
470058,Sadak 2,1.2,96986
131201,Superbabies: Baby Geniuses 2,1.5,32013
271390,Elk*rtuk,1.5,40838
416592,Kod Adi K.O.Z.,1.7,27604
249299,Birdemic: Shock and Terror,1.7,26385
33525,Manos: The Hands of Fate,1.7,38151
295378,Justin Bieber: Never Say Never,1.7,76514


In [23]:
#Filmes de TV
df_merge[['originalTitle', 'averageRating', 'numVotes']].loc[(df_merge.titleType == 'tvMovie') & (df_merge.numVotes >= 25000 )].sort_values(by = 'averageRating').head(10)

,originalTitle,averageRating,numVotes
150187,Home Alone 4: Taking Back the House,2.6,41906
355940,Sharknado,3.3,54906
293773,Mean Girls 2,4.1,26757
213384,Camp Rock,5.2,39357
194140,High School Musical 2,5.3,69428
185186,High School Musical,5.6,101492
233513,Princess Protection Program,5.7,27930
215756,Another Cinderella Story,5.8,32230
393265,Descendants,6.3,29183
310620,The Wizard of Lies,6.8,30941


## **TOP 10 Séries de TV com as melhores avaliações**
(25000 votos é o valor mínimo [necessário](https://en.wikipedia.org/wiki/IMDb) para estar listado entre os melhores no IMDb)

In [24]:
df_merge[['originalTitle', 'averageRating', 'numVotes']].loc[(df_merge.titleType == 'tvSeries') & (df_merge.numVotes >= 25000 )].sort_values(by = 'averageRating', ascending = False).head(10)

,originalTitle,averageRating,numVotes
200172,Breaking Bad,9.5,2345889
467460,Bluey,9.3,39212
172442,Avatar: The Last Airbender,9.3,408716
143393,The Wire,9.3,405405
373017,Sapne Vs Everyone,9.3,73988
266870,Aspirants,9.2,315883
485674,Our Planet,9.2,57193
201926,Game of Thrones,9.2,2444737
87422,The Sopranos,9.2,529189
41778,The World at War,9.2,33273


## **TOP 10 Séries de TV com as piores avaliações**

In [25]:
df_merge[['originalTitle', 'averageRating', 'numVotes']].loc[(df_merge.titleType == 'tvSeries') & (df_merge.numVotes >= 25000 )].sort_values(by = 'averageRating').head(10)

,originalTitle,averageRating,numVotes
263711,Velma,1.6,80624
217770,Keeping Up with the Kardashians,2.9,31986
479593,Batwoman,3.6,47520
489179,Resident Evil,4.2,47949
237099,The Acolyte,4.2,135422
274216,The Idol,4.4,114614
221438,Tandav,4.7,61493
411355,Inhumans,4.9,30033
268158,Raketsonyeondan,4.9,29491
313152,Indian Police Force,5.0,58566


## **Filmes com maior duração**

In [26]:
#Filmes
df_merge[['originalTitle', 'runtimeMinutes']].loc[df_merge.titleType == 'movie'].sort_values(by = 'runtimeMinutes', ascending = False).head(10)

,originalTitle,runtimeMinutes
368113,100,59460
474756,Logistics,51420
406060,Ambiancé,43200
352206,Modern Times Forever,14400
217442,Qw,10062
339628,Beijing 2003,9000
428194,London EC1,5460
136180,The Cure for Insomnia,5220
153498,The Longest Most Meaningless Movie in the World,2880
219331,Five-Year Diary,2160


In [27]:
#Filmes de TV
df_merge[['originalTitle', 'runtimeMinutes']].loc[df_merge.titleType == 'tvMovie'].sort_values(by = 'runtimeMinutes', ascending = False).head(10)

,originalTitle,runtimeMinutes
477846,24H Europe: The Next Generation,1440
245807,24h Berlin - Ein Tag im Leben,1440
408542,24h Jerusalem,1440
135647,Johann Wolfgang von Goethe: Faust II,1320
173630,The Bill @ 21,960
51853,Jordbrosviten,720
171957,War and Peace in the Nuclear Age,715
250095,The Third Day: Autumn,715
177594,Birth Day Live!,600
336571,Second Move Kills - 5 Jahre mit Jens Spahn,600
